In [10]:
# Import modules.
from sklearn.utils import shuffle
#from pysad.evaluation import AUROCMetric
from pysad.models import xStream
from pysad.utils import ArrayStreamer
from pysad.transform.postprocessing import RunningAveragePostprocessor
from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.utils import Data
from tqdm import tqdm
import numpy as np

from pysad.models.integrations.reference_window_model import ReferenceWindowModel
from pysad import models
from pyod.models.iforest import IForest

from pyod.models.iforest import IForest
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.utils import Data
from tqdm import tqdm
import numpy as np
import scipy.io
from river import drift
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os 
from hyperopt import fmin, tpe 

In [19]:
dataset ="dataset/nab-data/realTraffic/TravelTime_387.csv"
df = pd.read_csv(dataset)
#plot_time_series(df["value"].values,title=dataset).show()
column="value"
from hyperopt import hp, STATUS_OK, Trials
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
X =np.array([[i] for i in df[column].values])
iforestASD=class_iforestASD.iforestASD
right=[387,948,1485]
nbr_anomalies=3
gap =200
class_iforestASD.test(right,nbr_anomalies,gap)

"""scores=iforestASD(X,window_size=best["window_size"],n_estimators=best["n_estimators"])
print(scores)
plot_time_series(scores, title="scores").show()
"""

{'n_estimators': 95, 'window_size': 958}             
changed detected at                                  
1695                                                 
{'n_estimators': 17, 'window_size': 746}                                       
changed detected at                                                            
1727                                                                           
100%|██████████| 2/2 [02:09<00:00, 64.99s/trial, best loss: 0.6666666666666666]
{'n_estimators_index': 94, 'window_size_index': 758}
changed detected at 1695
the final score is 0.6666666666666666 [333, 989, 2041, 2042]


'scores=iforestASD(X,window_size=best["window_size"],n_estimators=best["n_estimators"])\nprint(scores)\nplot_time_series(scores, title="scores").show()\n'

In [18]:
class class_iforestASD:
    def __init__(nbr_anomalies,right,gap):
        self.nbr_anomalies= nbr_anomalies
        print("ok")
    def iforestASD(X,window_size,n_estimators=100):
        X =np.array(X)
        """
        Malheureusement le concept drift n'est pas encore implémenté dans pysad nous devons le faire manuellement
        n_estimators est le nombre d'arbres nécéssaire
        Window_size est la taille de la fenêtre
        """
        initial=window_size
        np.random.seed(61)  # Fix random seed.
        drift_detector = drift.ADWIN()
        X_all =X
        iterator = ArrayStreamer(shuffle=False)
        model=models.IForestASD(initial_window_X=X_all[:initial],window_size=window_size)
        model.fit(X_all[:window_size])
        model.n_estimators=100

        i=0
        scores=[]
        batchs=range (0,len(X_all),window_size)
        flag=False
        for batch in batchs:
            i+=1
            #score =model.score(X_all[batch:batch+window_size])
            if flag==True:
                model=models.IForestASD(initial_window_X=X_all[batch:batch+window_size],window_size=window_size)
                model.fit(X_all[batch:batch+window_size])
                flag=False
            #print(scores)
            score =model.score(X_all[batch:batch+window_size])
            for id,one_score in enumerate(score) :
                in_drift, in_warning =  drift_detector.update(one_score)
                if in_drift:
                    print("changed detected at", id+batch)
                    flag=True
                    break
            
            score =list(np.array(score))
            scores=scores+score
        return scores

    def test(right,nbr_anomalies,gap):

        #right=[387,948,1485]
        #nbr_anomalies=3
        def scoring(scores):
            score=0
            for real in right:
                if 1 in scores[real-gap:real+gap]:
                    score+=1
            score=score/nbr_anomalies
            return score
            
        def objective(args):
            print(args)
            scores= iforestASD(X,window_size=args["window_size"],n_estimators=args["n_estimators"]
                                )
            scores =score_to_label(nbr_anomalies,scores,gap)
            
            return scoring(scores)#{'loss': 1/1+score, 'status': STATUS_OK}


        possible_nbr_tree =[*range(1,100)]
        possible_window_size =[*range(200,1000)]
        space2 ={"window_size":hp.choice("window_size_index",possible_window_size)
        , "n_estimators":hp.choice("n_estimators_index",possible_nbr_tree)}
        trials = Trials()
        best = fmin(fn=objective,space=space2, algo=tpe.suggest, max_evals=2,trials = trials)
        print(best)
        real_scores= iforestASD(X,window_size=possible_window_size[best["window_size_index"]],n_estimators=possible_nbr_tree[best["n_estimators_index"]])
        scores =score_to_label(nbr_anomalies,real_scores,gap)
        identified =[key for key, val in enumerate(scores) if val in [1]] 
        print("the final score is", scoring(scores),identified)
        return real_scores, scores, identified,scoring(scores)



In [53]:

def check (indice, real_indices,gap):
    Flag=True
    for real_indice in real_indices:
        #print(indice, [*range(real_indice-gap,real_indice+gap)])
        if indice in [*range(real_indice-gap,real_indice+gap)]:
            Flag=False
    return Flag
def score_to_label(nbr_anomalies,scores,gap=200):
  gap=200
  threshold=0.00001
  tmp=scores.copy()
  indices=[]
  real_indices=[]
  while len(real_indices)<nbr_anomalies:
    threshold =max(tmp)
    index = tmp.index(max(tmp))
    del tmp[index]    
    
    indices= [i for i,val in enumerate(scores) if val==threshold] 
    for indice in indices:
        if check(indice,real_indices,gap):
            real_indices.append(indice)
        print("**",threshold,(real_indices))
  return [0 if i<threshold else 1 for i in scores ]


print(real_scores[428])


** 0.34994112147414735 [2305]
** 0.3403204796743634 [2305]
** 0.25683057266603193 [2305]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154]
** 0.21939367058592152 [2305, 154, 428]
** 0.21939367058592152 [2305, 154, 428, 989]
[154, 269, 270, 271, 333, 334, 335, 428, 989, 2304, 2305, 2436]
0.21939367058592152


In [20]:
"hhh".split(";")

['hhh']

In [31]:
# Test pipeline   les threshold des methodes coe iforest seront récupérés dans NAB parce qu'NAB à une fonction de score automatisé. 
import time
import os
base_file ='Pattern_lengths_and_Number_of_discords2.xlsx'
base = pd.read_excel(base_file)
methods= {"MILOF":MILOF,"iforestASD":iforestASD, "HStree":HStree } # "MILOF":MILOF


for key, method in methods.items():
  thresholds=[]
  for idx, dataset in enumerate(base["Dataset"]):
    try:
      final=base[key+":precision"]
      a= base[key+":precision"][idx]
      if a !=0:
        continue
    except Exception as e:
      print("An exception occurred: ", e)
      final =np.zeros(len(base["Dataset"]))
    print(dataset)
    if os.path.exists("real_nab_data/"+dataset) :
      start =time.monotonic()
      df = pd.read_csv("real_nab_data/"+dataset)
      column="value"
      X =[[i] for i in df[column].values]
      dataset =dataset.replace('nab-data/','')
      directory = os.path.dirname(('streaming_results/'+key+'/'+key+'/'+dataset))
      if not os.path.exists(directory):
        print("pbssss")
      data_file_name=dataset.split('/')[-1]
      data_file_name =key+'_'+data_file_name
      dataset =directory+'/'+data_file_name
      print(dataset)
      try:
          df=pd.read_csv(dataset)
      except Exception as e:
        continue 
      #threshold= base[key+"_threshold"][idx]
      scores=list(df["anomaly_score"].values)
      nbr_anomalies=len(base["Position discord"][idx].split(';'))
      tmp=scores.copy()
      print(type(tmp))
      threshold = 0.000001
      predicted=[]
      expected =[]
      for i in range(nbr_anomalies):
        if (max(tmp)>threshold):
          threshold =max(tmp)
        index = tmp.index(max(tmp))
        tmp[index]=0
        predicted.append(int(index))
      for ano in base["Position discord"][idx].split(';') :
          expected =expected+[*range(int(ano)-int(len(X)/100)),int(ano)+int(len(X)/100)]
      print(np.array(expected),np.array(predicted))
      overlap=overlapping_rate(predicted, expected)/len(base["Position discord"][idx].split(';'))
      final[idx]=overlap
      base[key+":precision"]=final
      base.to_excel("Pattern_lengths_and_Number_of_discords2.xlsx")


  


  
print("terminé")

stdb_308_1.txt
stdb_308_2.txt
xmitdb_x108_1.txt
xmitdb_x108_2.txt
mitdb__100_180_1.txt
mitdb__100_180_2.txt
chfdb_chf01_275_1.txt
chfdb_chf01_275_2.txt
ltstdb_20221_43_1.txt
ltstdb_20221_43_2.txt
ltstdb_20321_240_1.txt
ltstdb_20321_240_2.txt
chfdb_chf13_45590_1.txt
chfdb_chf13_45590_2.txt
mitdbx_mitdbx_108.txt
qtdbsele0606.txt
chfdbchf15.txt
qtdbsel102.txt
ann_gun_CentroidA_1.txt
Marotta_Valve_Tek_16.txt
Marotta_Valve_Tek_17.txt
Marotta_Valve_Tek_14.txt
Patient_respiration.txt
Patient_respiration2.txt
dutch_power_demand.txt
taxi.txt
GPS_trajectory_data.csv
nab-data/artificialWithAnomaly/art_daily_flatmiddle.csv
streaming_results/MILOF/MILOF/artificialWithAnomaly/MILOF_art_daily_flatmiddle.csv
<class 'list'>
[   0    1    2 ... 2637 2638 2719] [3960]
the percentage is 0
terminé
nab-data/artificialWithAnomaly/art_daily_nojump.csv
streaming_results/MILOF/MILOF/artificialWithAnomaly/MILOF_art_daily_nojump.csv
<class 'list'>
[   0    1    2 ... 2745 2746 2827] [3197]
the percentage is 0
ter

/home/amngobibin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



nab-data/realAWSCloudwatch/ec2_cpu_utilization_24ae8d.csv
streaming_results/MILOF/MILOF/realAWSCloudwatch/MILOF_ec2_cpu_utilization_24ae8d.csv
<class 'list'>
[   0    1    2 ... 3635 3636 3717] [0 0]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_cpu_utilization_53ea38.csv
streaming_results/MILOF/MILOF/realAWSCloudwatch/MILOF_ec2_cpu_utilization_53ea38.csv
<class 'list'>
[   0    1    2 ... 2517 2518 2599] [0 0]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_cpu_utilization_5f5533.csv
streaming_results/MILOF/MILOF/realAWSCloudwatch/MILOF_ec2_cpu_utilization_5f5533.csv
<class 'list'>
[   0    1    2 ... 2788 2789 2870] [ 247 3448]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_cpu_utilization_77c1ca.csv
streaming_results/MILOF/MILOF/realAWSCloudwatch/MILOF_ec2_cpu_utilization_77c1ca.csv
<class 'list'>
[   0    1    2 ... 1723 1724 1805] [2306]
the percentage is 0
terminé
nab-data/realAWSCloudwatch/ec2_network_in_5abac7.csv
streaming_results/MILOF/MILO

nab-data/realTraffic/occupancy_t4013.csv
streaming_results/MILOF/MILOF/realTraffic/MILOF_occupancy_t4013.csv
<class 'list'>
[   0    1    2 ... 2308 2309 2360] [0 0]
the percentage is 1
terminé
nab-data/realTraffic/speed_7578.csv
streaming_results/MILOF/MILOF/realTraffic/MILOF_speed_7578.csv
<class 'list'>
[  0   1   2 ... 932 933 956] [0 0 0 0]
the percentage is 1
terminé
nab-data/realTraffic/speed_t4013.csv
streaming_results/MILOF/MILOF/realTraffic/MILOF_speed_t4013.csv
<class 'list'>
[   0    1    2 ... 2309 2310 2359] [0 0]
the percentage is 1
terminé
nab-data/realTweets/Twitter_volume_AMZN.csv
streaming_results/MILOF/MILOF/realTweets/MILOF_Twitter_volume_AMZN.csv
<class 'list'>
[    0     1     2 ... 11249 11250 11567] [0 0 0 0]
the percentage is 1
terminé
nab-data/realTweets/Twitter_volume_CRM.csv
streaming_results/MILOF/MILOF/realTweets/MILOF_Twitter_volume_CRM.csv
<class 'list'>
[   0    1    2 ... 7607 7608 7927] [0 0 0]
the percentage is 1
terminé
nab-data/realTweets/Twitter_

nab-data/realAdExchange/exchange-2_cpm_results.csv
streaming_results/iforestASD/iforestASD/realAdExchange/iforestASD_exchange-2_cpm_results.csv
<class 'list'>
[  0   1   2 ... 916 917 950] [1013 1184]
the percentage is 0
terminé
nab-data/realAdExchange/exchange-3_cpc_results.csv
streaming_results/iforestASD/iforestASD/realAdExchange/iforestASD_exchange-3_cpc_results.csv
<class 'list'>
[  0   1   2 ... 935 936 967] [1121  976  979]
the percentage is 0
terminé
nab-data/realAdExchange/exchange-3_cpm_results.csv
streaming_results/iforestASD/iforestASD/realAdExchange/iforestASD_exchange-3_cpm_results.csv
<class 'list'>
[   0    1    2 ... 1028 1029 1060] [1121]
the percentage is 0
terminé
nab-data/realKnownCause/ec2_request_latency_system_failure.csv
streaming_results/iforestASD/iforestASD/realKnownCause/iforestASD_ec2_request_latency_system_failure.csv
<class 'list'>
[   0    1    2 ... 3914 3915 3996] [4024 4026 4030]
the percentage is 0
terminé
nab-data/realKnownCause/rogue_agent_key_hol

nab-data/realAWSCloudwatch/ec2_disk_write_bytes_1ef3de.csv
streaming_results/HStree/HStree/realAWSCloudwatch/HStree_ec2_disk_write_bytes_1ef3de.csv
<class 'list'>
[   0    1    2 ... 2350 2351 2446] [461]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_disk_write_bytes_c0d644.csv
streaming_results/HStree/HStree/realAWSCloudwatch/HStree_ec2_disk_write_bytes_c0d644.csv
<class 'list'>
[   0    1    2 ... 2922 2923 3004] [1499 1499 1499]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_network_in_257a54.csv
streaming_results/HStree/HStree/realAWSCloudwatch/HStree_ec2_network_in_257a54.csv
<class 'list'>
[   0    1    2 ... 1395 1396 1477] [0]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/ec2_network_in_5abac7.csv
streaming_results/HStree/HStree/realAWSCloudwatch/HStree_ec2_network_in_5abac7.csv
<class 'list'>
[   0    1    2 ... 3042 3043 3138] [0 0]
the percentage is 1
terminé
nab-data/realAWSCloudwatch/elb_request_count_8c0756.csv
streaming_results/HStree/HS

nab-data/realTraffic/occupancy_t4013.csv
streaming_results/HStree/HStree/realTraffic/HStree_occupancy_t4013.csv
<class 'list'>
[   0    1    2 ... 2308 2309 2360] [2359 2359]
the percentage is 0
terminé
nab-data/realTraffic/speed_6005.csv
streaming_results/HStree/HStree/realTraffic/HStree_speed_6005.csv
<class 'list'>
[   0    1    2 ... 2234 2235 2286] [0]
the percentage is 1
terminé
nab-data/realTraffic/speed_7578.csv
streaming_results/HStree/HStree/realTraffic/HStree_speed_7578.csv
<class 'list'>
[  0   1   2 ... 932 933 956] [0 0 0 0]
the percentage is 1
terminé
nab-data/realTraffic/speed_t4013.csv
streaming_results/HStree/HStree/realTraffic/HStree_speed_t4013.csv
<class 'list'>
[   0    1    2 ... 2309 2310 2359] [0 0]
the percentage is 1
terminé
nab-data/realTweets/Twitter_volume_AAPL.csv
streaming_results/HStree/HStree/realTweets/HStree_Twitter_volume_AAPL.csv
<class 'list'>
[   0    1    2 ... 8926 8927 9246] [3568 3568 3568 3568]
the percentage is 1
terminé
nab-data/realTweets